In [ ]:
import daily_snapshot_comparison as dsc
# import daily_snapshot_extraction as dse
import os
from tqdm.notebook import tqdm
import pandas as pd
import getpass

import warnings
warnings.filterwarnings("ignore")
                        
# Set the display options
pd.options.display.float_format = '{:,.2f}'.format


%load_ext autoreload
%autoreload 2

%reload_ext autoreload

#### Pseudocode


Steps
- takes parameter on what type of analysis
    - load past 8 days of data from folders
    - get columns required
    - get last day snapshot
    - get time-frame
    - compare values
    - create summary report
    - save into excel

In [70]:
# Parameters
folder = r'C:/Users/SESA676486/OneDrive - Schneider Electric/7) Data Quality and Support Centre/02. Finance RDO/03. Data Discovery/Daily Snapshots'
user = input('SESAID: ')
password = getpass.getpass("Password for " + user + ":")
file_limit = 8
report_date = dsc.date_time_now()
last_year_month = '202406'
time_frames = ['No Filter', 'YTD']

# Conditions for outliers
gap_upper = 3_000_000.0
gap_lower = -1_000_000.0
pct_gap_upper = 0.15
pct_gap_lower = -0.15

# summary report
top_count = 5


daily_orders_dict = {
    'net_orders_re': {'source': 'net_orders_re_pl', 'field_list': ['extract date', 'se_rpg_entity'], 'value_list': ['std_net_price_hfm_rate']},
    'net_orders_pl': {'source': 'net_orders_re_pl', 'field_list': ['extract date', 'product_line_code'], 'value_list': ['std_net_price_hfm_rate']},
    'net_orders_re_pl': {'source': 'net_orders_re_pl', 'field_list': ['extract date', 'se_rpg_entity', 'product_line_code'], 'value_list': ['std_net_price_hfm_rate']},
    'net_orders_gdp': {'source': 'net_orders_gdp_offer', 'field_list': ['extract date', 'gdp_code'], 'value_list': ['std_net_price_hfm_rate']},
    'net_orders_cr': {'source': 'net_orders_cr_gdp', 'field_list': ['extract date', 'gdp_code', 'finance_commercial_reference'], 'value_list': ['std_net_price_hfm_rate']},
    'net_orders_offer': {'source': 'net_orders_gdp_offer', 'field_list': ['extract date', 'orh_offer_owner_label'], 'value_list': ['std_net_price_hfm_rate']},
    'net_orders_connectable': {'source': 'net_orders_gdp_offer', 'field_list': ['extract date', 'connectable'], 'value_list': ['std_net_price_hfm_rate']},
    'net_orders_recurrence': {'source': 'net_orders_gdp_offer', 'field_list': ['extract date', 'reccurence_label'], 'value_list': ['std_net_price_hfm_rate']},
    'net_orders_bus': {'source': 'net_orders_gdp_offer', 'field_list': ['extract date', 'business_model'], 'value_list': ['std_net_price_hfm_rate']},
    'net_orders_layer':  {'source': 'net_orders_gdp_offer', 'field_list': ['extract date', 'df_layer_tag'], 'value_list': ['std_net_price_hfm_rate']}
}

daily_invoices_dict = {
    'invoice_sales_re': {'source': 'invoice_sales_re_pl', 'field_list': ['extract date', 'se_rpg_entity'], 'value_list': ['net_amount_invoiced_hfm_rate']},
    'invoice_sales_pl': {'source': 'invoice_sales_re_pl', 'field_list': ['extract date', 'product_line_code'], 'value_list': ['net_amount_invoiced_hfm_rate']},
    'invoice_sales_re_pl': {'source': 'invoice_sales_re_pl', 'field_list': ['extract date', 'se_rpg_entity', 'product_line_code'], 'value_list': ['net_amount_invoiced_hfm_rate']},
    'invoice_sales_gdp': {'source': 'invoice_sales_gdp_offer', 'field_list': ['extract date', 'gdp_code'], 'value_list': ['net_amount_invoiced_hfm_rate']},
    'invoice_sales_cr': {'source': 'invoice_sales_cr_gdp', 'field_list': ['extract date', 'gdp_code', 'finance_commercial_reference'], 'value_list': ['net_amount_invoiced_hfm_rate']},
    'invoice_sales_offer': {'source': 'invoice_sales_gdp_offer', 'field_list': ['extract date', 'orh_offer_owner_label'], 'value_list': ['net_amount_invoiced_hfm_rate']},
    'invoice_sales_connectable': {'source': 'invoice_sales_gdp_offer', 'field_list': ['extract date', 'connectable'], 'value_list': ['net_amount_invoiced_hfm_rate']},
    'invoice_sales_recurrence': {'source': 'invoice_sales_gdp_offer', 'field_list': ['extract date', 'reccurence_label'], 'value_list': ['net_amount_invoiced_hfm_rate']},
    'invoice_sales_bus': {'source': 'invoice_sales_gdp_offer', 'field_list': ['extract date', 'business_model'], 'value_list': ['net_amount_invoiced_hfm_rate']},
    'invoice_sales_layer':  {'source': 'invoice_sales_gdp_offer', 'field_list': ['extract date', 'df_layer_tag'], 'value_list': ['net_amount_invoiced_hfm_rate']}
}


brcc_orders_dict = {
    'brcc_orders_pl': {'source': 'net_orders_BRCC', 'field_list': ['extract date', 'pm0_product_line_note'], 'value_list': ['std_net_price_hfm_rate']},
    'brcc_orders_gdp': {'source': 'net_orders_BRCC', 'field_list': ['extract date', 'gdp_code'], 'value_list': ['std_net_price_hfm_rate']},
    'brcc_orders_cr': {'source': 'net_orders_BRCC', 'field_list': ['extract date', 'finance_commercial_reference'], 'value_list': ['std_net_price_hfm_rate']},
    'brcc_orders_recurrence': {'source': 'net_orders_BRCC', 'field_list': ['extract date', 'reccurence_label'], 'value_list': ['std_net_price_hfm_rate']},
    'brcc_orders_layer':  {'source': 'net_orders_BRCC', 'field_list': ['extract date', 'df_layer_tag'], 'value_list': ['std_net_price_hfm_rate']}
}


brcc_invoices_dict = {
    'brcc_orders_pl': {'source': 'invoice_sales_BRCC', 'field_list': ['extract date', 'pm0_product_line_note'], 'value_list': ['net_amount_invoiced_hfm_rate']},
    'brcc_orders_gdp': {'source': 'invoice_sales_BRCC', 'field_list': ['extract date', 'gdp_code'], 'value_list': ['net_amount_invoiced_hfm_rate']},
    'brcc_orders_cr': {'source': 'invoice_sales_BRCC', 'field_list': ['extract date', 'finance_commercial_reference'], 'value_list': ['net_amount_invoiced_hfm_rate']},
    'brcc_orders_recurrence': {'source': 'invoice_sales_BRCC', 'field_list': ['extract date', 'reccurence_label'], 'value_list': ['net_amount_invoiced_hfm_rate']},
    'brcc_orders_layer':  {'source': 'invoice_sales_BRCC', 'field_list': ['extract date', 'df_layer_tag'], 'value_list': ['net_amount_invoiced_hfm_rate']}
}

In [64]:
# loop over items in daily_orders_dict to do daily snapshots
summary_cols = ['attribute_label','attribute_value', 'tag',  'time_frame','date_prev','date_curr', 'amt_prev', 'amt_curr', 'gap', 'pct_gap']

summary_df = pd.DataFrame(columns=summary_cols)

for row_dict, query in daily_orders_dict.items():

    df_combined = dsc.combined_csv(folder + '/data', query['source'], file_limit)
    
    for t in time_frames:
        
        df_time_frame = dsc.time_frame(df_combined, time_frame=t, last_year_month=last_year_month)
        
        df_grouped = dsc.group_data(df_time_frame, query['field_list'], query['value_list'])
        
        df_trended = dsc.trend_data(df_grouped, query['field_list'][1:], query['value_list'], query['field_list'][0])
        
        df_gap = dsc.gap(df_trended, file_limit)
        
        df_flagged = dsc.flag_outliers(df_trended, df_gap, file_limit, gap_upper, gap_lower, pct_gap_upper, pct_gap_lower)
        
        df_summarized = dsc.summarize_outliers(df_trended, df_flagged, t, query['field_list'][1:],  file_limit, top_count)

        summary_df = summary_df.append(df_summarized)

values_to_keep = ['se_rpg_entity', 'product_line_code', 'gdp_code', 'finance_commercial_reference',
                  'orh_offer_owner_label', 'connectable', 'reccurence_label', 'business_model', 'df_layer_tag']

summary_df = summary_df[summary_df['attribute_label'].isin(values_to_keep)]

summary_df.to_excel(f'{folder}/summary/{report_date}_net_orders_summary.xlsx', index=False)

In [65]:
# loop over items in daily_invoices_dict to do daily snapshots
summary_cols = ['attribute_label','attribute_value', 'tag',  'time_frame','date_prev','date_curr', 'amt_prev', 'amt_curr', 'gap', 'pct_gap']

summary_df = pd.DataFrame(columns=summary_cols)

for row_dict, query in daily_invoices_dict.items():

    df_combined = dsc.combined_csv(folder + '/data', query['source'], file_limit)
    
    for t in time_frames:
        
        df_time_frame = dsc.time_frame(df_combined, time_frame=t, last_year_month=last_year_month)
        
        df_grouped = dsc.group_data(df_time_frame, query['field_list'], query['value_list'])
        
        df_trended = dsc.trend_data(df_grouped, query['field_list'][1:], query['value_list'], query['field_list'][0])
        
        df_gap = dsc.gap(df_trended, file_limit)
        
        df_flagged = dsc.flag_outliers(df_trended, df_gap, file_limit, gap_upper, gap_lower, pct_gap_upper, pct_gap_lower)
        
        df_summarized = dsc.summarize_outliers(df_trended, df_flagged, t, query['field_list'][1:],  file_limit, top_count)

        summary_df = summary_df.append(df_summarized)

values_to_keep = ['se_rpg_entity', 'product_line_code', 'gdp_code', 'finance_commercial_reference',
                  'orh_offer_owner_label', 'connectable', 'reccurence_label', 'business_model', 'df_layer_tag']

summary_df = summary_df[summary_df['attribute_label'].isin(values_to_keep)]

summary_df.to_excel(f'{folder}/summary/{report_date}_invoice_sales_summary.xlsx', index=False)

In [75]:
summary_cols = ['attribute_label','attribute_value', 'tag',  'time_frame','date_prev','date_curr', 'amt_prev', 'amt_curr', 'gap', 'pct_gap']

summary_df = pd.DataFrame(columns=summary_cols)

for row_dict, query in brcc_orders_dict.items():

    df_combined = dsc.combined_csv(folder + '/data', query['source'], 2)
    
    for t in time_frames:
        
        df_time_frame = dsc.time_frame(df_combined, time_frame=t, last_year_month=last_year_month)
        
        df_grouped = dsc.group_data(df_time_frame, query['field_list'], query['value_list'])
        
        df_trended = dsc.trend_data(df_grouped, query['field_list'][1:], query['value_list'], query['field_list'][0])
        
        df_gap = dsc.gap(df_trended, 2)
        
        df_flagged = dsc.flag_outliers(df_trended, df_gap, 2, gap_upper, gap_lower, pct_gap_upper, pct_gap_lower)
        df_flagged.to_csv(f'brcc_orders_{row_dict}_{t}.csv', index=False)
        
        df_summarized = dsc.summarize_outliers(df_trended, df_flagged, t, query['field_list'][1:],  2, top_count)

        summary_df = summary_df.append(df_summarized)

values_to_keep = ['pm0_product_line_note', 'gdp_code', 'finance_commercial_reference', 'reccurence_label', 'df_layer_tag']

summary_df = summary_df[summary_df['attribute_label'].isin(values_to_keep)]

summary_df.to_csv('brcc_orders_summary.csv', index=False)

In [76]:
summary_cols = ['attribute_label','attribute_value', 'tag',  'time_frame','date_prev','date_curr', 'amt_prev', 'amt_curr', 'gap', 'pct_gap']

summary_df = pd.DataFrame(columns=summary_cols)

for row_dict, query in brcc_invoices_dict.items():

    df_combined = dsc.combined_csv(folder + '/data', query['source'], 2)
    
    for t in time_frames:
        
        df_time_frame = dsc.time_frame(df_combined, time_frame=t, last_year_month=last_year_month)
        
        df_grouped = dsc.group_data(df_time_frame, query['field_list'], query['value_list'])
        
        df_trended = dsc.trend_data(df_grouped, query['field_list'][1:], query['value_list'], query['field_list'][0])
        
        df_gap = dsc.gap(df_trended, 2)
        
        df_flagged = dsc.flag_outliers(df_trended, df_gap, 2, gap_upper, gap_lower, pct_gap_upper, pct_gap_lower)
        df_flagged.to_csv(f'brcc_invoices_{row_dict}_{t}.csv', index=False)
        
        df_summarized = dsc.summarize_outliers(df_trended, df_flagged, t, query['field_list'][1:],  2, top_count)

        summary_df = summary_df.append(df_summarized)

values_to_keep = ['pm0_product_line_note', 'gdp_code', 'finance_commercial_reference', 'reccurence_label', 'df_layer_tag']

summary_df = summary_df[summary_df['attribute_label'].isin(values_to_keep)]

summary_df.to_csv('brcc_invoices_summary.csv', index=False)